In [4]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the JSON data
file_name = 'resultados_avaliacoes.json'
with open(file_name, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)

# Separate into individual evaluations and BERT score comparisons
df_individual = df[df['tipo_de_registro'] == 'avaliacao_individual'].copy()
df_bertscore = df[df['tipo_de_registro'] == 'comparacao_bertscore'].copy()

# --- Data Cleaning and Preparation for df_individual ---
# Convert score_final to numeric
df_individual['score_final'] = pd.to_numeric(df_individual['score_final'])

# Extract LLM evaluation scores
llm_metrics = ['fidelidade', 'relevancia', 'completude', 'verificacao_seguranca']
for metric in llm_metrics:
    df_individual[f'{metric}_pontuacao'] = df_individual['avaliacoes_llm'].apply(lambda x: x[metric]['pontuacao'] if isinstance(x, dict) and metric in x and isinstance(x[metric], dict) else np.nan)
    df_individual[f'{metric}_pontuacao'] = pd.to_numeric(df_individual[f'{metric}_pontuacao'])

# --- Data Cleaning and Preparation for df_bertscore ---
df_bertscore['bertscore_f1'] = df_bertscore['similaridade_bertscore'].apply(lambda x: x['bertscore_f1'] if isinstance(x, dict) and 'bertscore_f1' in x else np.nan)
df_bertscore['bertscore_f1'] = pd.to_numeric(df_bertscore['bertscore_f1'])

# Define the color palette
PALETA_SUAVE = [
    "#56C6EF",  # azul
    "#C0A8FF",  # lilás
    "#F981A9",  # rosa
    "#8FD7F2",  # azul claro
    "#C8C8F4",  # lilás claro
    "#F5B4DE"   # rosa claro
]

# Adjust Matplotlib style for minimalism
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['axes.edgecolor'] = 'lightgray'
plt.rcParams['axes.linewidth'] = 0.8
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = '#EEEEEE'
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['xtick.color'] = 'dimgray'
plt.rcParams['ytick.color'] = 'dimgray'
plt.rcParams['axes.labelcolor'] = 'dimgray'
plt.rcParams['axes.titlecolor'] = 'dimgray'
plt.rcParams['figure.titlesize'] = 16
plt.rcParams['axes.titlesize'] = 'large'
plt.rcParams['axes.labelsize'] = 'small'
plt.rcParams['axes.labelsize'] = 12  # Tamanho da fonte para os rótulos dos eixos (ex: 'Score Final')
plt.rcParams['xtick.labelsize'] = 10  # Tamanho da fonte para os números no eixo X
plt.rcParams['ytick.labelsize'] = 10  # Tamanho da fonte para os números no eixo Y
plt.rcParams['legend.fontsize'] = 11  # Tamanho da fonte para a legenda
plt.rcParams['legend.frameon'] = False
plt.rcParams['patch.edgecolor'] = 'none'

# --- Analysis and Plotting ---
plot_files = []

# Função para salvar os gráficos com alta qualidade
def salvar_grafico(file_path):
    """Salva a figura atual com alta resolução e bordas ajustadas."""
    # Usamos dpi=300 para alta qualidade e bbox_inches='tight' para remover margens.
    plt.savefig(file_path, dpi=300, bbox_inches='tight')
    plot_files.append(file_path)
    plt.close()

# 1. Distribution of Final Scores (score_final) ---------------------------------------------------------------------------------------------------------------
plt.figure(figsize=(8, 5))

# Ao chamar o plot, ele retorna o objeto Axes (eixos), vamos armazená-lo em 'ax'
ax = df_individual['score_final'].plot(kind='hist', bins=10, color=PALETA_SUAVE[0], rwidth=0.9)

# Usar 'ax' para modificar os spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# plt.title('Distribuição dos Scores Finais (Avaliação Individual)', pad=20)
plt.xlabel('Score Final')
plt.ylabel('Frequência')

if ax.containers: # Verifica se o container de barras foi criado
    ax.bar_label(
        ax.containers[0],     # Pega o primeiro (e geralmente único) container de barras
        fmt='%d',             # Formata o número como inteiro (sem casas decimais)
        label_type='edge',    # Coloca o rótulo na borda superior da barra
        padding=3,            # Pequeno espaçamento (em pontos) acima da barra
        fontsize=8            # Tamanho da fonte dos rótulos (ajuste conforme necessário)
    )
# Ajustar os limites do eixo Y para garantir que os rótulos caibam, se necessário
current_ylim = ax.get_ylim()
ax.set_ylim(current_ylim[0], current_ylim[1] * 1.05) # Aumenta o limite superior em 5%


plt.tight_layout()
salvar_grafico('./graphs/distribuicao_scores_finais.png')

# 2. Average Final Score by RAG (modelo_rag) -------------------------------------------------------------------------------------------------------------------
avg_score_by_model = df_individual.groupby('modelo_rag')['score_final'].mean().sort_values()
plt.figure(figsize=(10, 6))

# Capturar o objeto Axes na variável 'ax'
ax = avg_score_by_model.plot(kind='barh', color=PALETA_SUAVE[1])

# Usar 'ax' para modificar os spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# plt.title('Score Final Médio por RAG', pad=20) # Título continua comentado
plt.xlabel('Score Final Médio')
plt.ylabel('RAG')

# --- Adicionando os valores nas barras ---
if ax.containers: # Verifica se o container de barras foi criado
    ax.bar_label(
        ax.containers[0],
        fmt='%.2f',          # Formata o número para exibir duas casas decimais
        label_type='edge',   # Coloca o rótulo na borda (extremidade direita) da barra
        padding=3,           # Pequeno espaçamento (em pontos) à direita da barra
        fontsize=8           # Tamanho da fonte dos rótulos (ajuste conforme necessário)
    )

current_xlim = ax.get_xlim()
padding_factor = 0.08  # 8% de padding
range_x = current_xlim[1] - current_xlim[0]
ax.set_xlim(current_xlim[0], current_xlim[1] + range_x * padding_factor)


plt.tight_layout()
salvar_grafico('./graphs/score_medio_por_rag.png')

# 3. Average Final Score by Question ID (id_pergunta) -------------------------------------------------------------------------------------------------------------------
avg_score_by_question = df_individual.groupby('id_pergunta')['score_final'].mean().sort_values()
plt.figure(figsize=(10, 7))

# Capturar o objeto Axes na variável 'ax_question'
ax_question = avg_score_by_question.plot(kind='barh', color=PALETA_SUAVE[2])

# Usar 'ax' para modificar os spines
ax_question.spines['right'].set_visible(False)
ax_question.spines['top'].set_visible(False)

# plt.title('Score Final Médio por ID da Pergunta', pad=20) # Título continua comentado
plt.xlabel('Score Final Médio')
plt.ylabel('ID da Pergunta')

# --- Adicionando os valores nas barras (para o gráfico de score por pergunta) ---
if ax_question.containers:
    ax_question.bar_label(
        ax_question.containers[0],
        fmt='%.2f',          # Formata o número para exibir duas casas decimais
        label_type='edge',   # Coloca o rótulo na borda (extremidade direita) da barra
        padding=3,           # Pequeno espaçamento (em pontos) à direita da barra
        fontsize=8           # Tamanho da fonte dos rótulos
    )

# Ajustar os limites do eixo X para garantir que os rótulos caibam
current_xlim_q = ax_question.get_xlim()
range_x_q = current_xlim_q[1] - current_xlim_q[0]
padding_factor_q = 0.08 # 8% de padding, ajuste se necessário
ax_question.set_xlim(current_xlim_q[0], current_xlim_q[1] + range_x_q * padding_factor_q)

plt.tight_layout()
salvar_grafico('./graphs/score_medio_por_pergunta.png')


# 4. Distribution of LLM Evaluation Metrics -------------------------------------------------------------------------------------------------------------------
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
# fig.suptitle('Distribuição das Pontuações das Métricas de Avaliação LLM', fontsize=16, y=1.03)

metrics_to_plot = {
    'fidelidade_pontuacao': 'Fidelidade',
    'relevancia_pontuacao': 'Relevância',
    'completude_pontuacao': 'Completude',
    'verificacao_seguranca_pontuacao': 'Verificação de Segurança'
}


for i, (metric_col, metric_name) in enumerate(metrics_to_plot.items()):
    ax_hist = axes[i//2, i%2] # 'ax_hist' refere-se ao subplot atual
    df_individual[metric_col].plot(kind='hist', bins=5, ax=ax_hist, color=PALETA_SUAVE[i%len(PALETA_SUAVE)], rwidth=0.9)
    ax_hist.set_title(metric_name)
    ax_hist.set_xlabel('Pontuação')
    ax_hist.set_ylabel('Frequência')
    ax_hist.spines['right'].set_visible(False)
    ax_hist.spines['top'].set_visible(False)

    # --- Adicionando os valores em cima das barras (para cada histograma) ---
    if ax_hist.containers: # Verifica se o container de barras foi criado no subplot
        ax_hist.bar_label(
            ax_hist.containers[0],
            fmt='%d',
            label_type='edge',
            padding=3,
            fontsize=8
        )
    # Ajustar os limites do eixo Y de cada subplot para garantir que os rótulos caibam
    current_ylim_hist = ax_hist.get_ylim()
    ax_hist.set_ylim(current_ylim_hist[0], current_ylim_hist[1] * 1.05) # Aumenta o limite superior em 5%

plt.tight_layout(rect=[0, 0, 1, 0.98])
salvar_grafico('./graphs/distribuicao_metricas_llm.png')



# 5. Average LLM Metric Scores by RAG -------------------------------------------------------------------------------------------------------------------
avg_metrics_by_model = df_individual.groupby('modelo_rag')[list(metrics_to_plot.keys())].mean()
avg_metrics_by_model.columns = [metrics_to_plot[col] for col in avg_metrics_by_model.columns]

PALETA_SUAVE_SCORES = [
    "#00BBFF",
    "#AE8EFF", 
    "#9FEAFF", 
    "#FFBCD5",  
    "#E4E1FF",
    "#F7AADC" 
]

plt.figure(figsize=(12, 7))
avg_metrics_by_model.T.plot(kind='bar', ax=plt.gca(), color=PALETA_SUAVE_SCORES)

# plt.title('Pontuação Média das Métricas LLM por RAG', pad=20)
plt.xlabel('Métrica de Avaliação')
plt.ylabel('Pontuação Média')
plt.xticks(rotation=45, ha='right')
plt.legend(
    title='RAG',
    loc='best',           # Mantém a lógica de encontrar o melhor lugar
    markerscale=0.7,      # Diminui os quadrados para 70% do tamanho original
    handletextpad=0.5,    # Reduz o espaço entre o quadrado e o texto
    fontsize='small'      # Diminui um pouco a fonte do texto da legenda
)

ax_com_rotulos = plt.gca()
# Usar 'ax' para modificar os spines
ax_com_rotulos.spines['right'].set_visible(False)
ax_com_rotulos.spines['top'].set_visible(False)
if ax_com_rotulos.containers: # Garante que existem barras (containers) para rotular
    for container in ax_com_rotulos.containers:
        ax_com_rotulos.bar_label(
            container,
            fmt='%.2f',          # Formato para duas casas decimais
            label_type='edge',   # Coloca o rótulo na borda superior da barra
            padding=3,           # Pequeno espaçamento (em pontos) acima da barra
            fontsize=7           # Tamanho da fonte menor para melhor encaixe
        )

    current_ylim = ax_com_rotulos.get_ylim()
    # Aumenta o limite superior do eixo Y em 8% (pode precisar de ajuste)
    ax_com_rotulos.set_ylim(current_ylim[0], current_ylim[1] * 1.08)

plt.tight_layout() # Mantido como no seu original
salvar_grafico('./graphs/metricas_llm_por_rag.png')

# 6. Average BERTScore F1 by Comparison Pair -------------------------------------------------------------------------------------------------------------------
avg_bertscore_by_comparison = df_bertscore.groupby('comparacao')['bertscore_f1'].mean().sort_values()
plt.figure(figsize=(12, 8))
avg_bertscore_by_comparison.plot(kind='barh', color=PALETA_SUAVE[4])

ax = plt.gca()

# Usar 'ax' para modificar os spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# plt.title('BERTScore F1 Médio por Par de Comparação', pad=20)
plt.xlabel('BERTScore F1 Médio')
plt.ylabel('Par de Comparação')

if ax.containers: # Garante que existem barras (containers) para rotular
    ax.bar_label(
        ax.containers[0],    # Pega o primeiro (e geralmente único) container de barras
        fmt='%.3f',          # Formata o número para exibir três casas decimais (comum para BERTScore F1)
        label_type='edge',   # Coloca o rótulo na borda (extremidade direita) da barra
        padding=3,           # Pequeno espaçamento (em pontos) à direita da barra
        fontsize=8           # Tamanho da fonte dos rótulos
    )

    current_xlim = ax.get_xlim()
    range_x = current_xlim[1] - current_xlim[0]
    ax.set_xlim(current_xlim[0], current_xlim[1] + range_x * 0.10)

plt.tight_layout()
salvar_grafico('./graphs/bertscore_f1_por_comparacao.png')

# 7. Count of Evaluations per Question ID -------------------------------------------------------------------------------------------------------------------
question_counts = df_individual['id_pergunta'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
question_counts.plot(kind='bar', color=PALETA_SUAVE[2])

ax = plt.gca()

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.title('Número de Avaliações por ID da Pergunta', pad=20)
plt.xlabel('ID da Pergunta')
plt.ylabel('Número de Avaliações')
plt.xticks(rotation=0)
plt.tight_layout()
salvar_grafico('./graphs/contagem_avaliacoes_por_pergunta.png')

# 8. Count of Evaluations per RAG -------------------------------------------------------------------------------------------------------------------
model_counts = df_individual['modelo_rag'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
model_counts.plot(kind='bar', color=PALETA_SUAVE[1])

ax = plt.gca()

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.title('Número de Avaliações por RAG', pad=20)
plt.xlabel('RAG')
plt.ylabel('Número de Avaliações')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
salvar_grafico('./graphs/contagem_avaliacoes_por_rag.png')

# 9: Performance (Média) vs. Consistência (Desvio Padrão) -------------------------------------------------------------------------------------------------------------------
print("Gerando Gráfico 9: Performance vs. Consistência com Quadrantes...")

# Calcular média e desvio padrão do score final para cada RAG
model_performance = df_individual.groupby('modelo_rag')['score_final'].agg(['mean', 'std']).reset_index()

plt.figure(figsize=(12, 8))

sns.scatterplot(
    x='mean',
    y='std',
    hue='modelo_rag',
    data=model_performance,
    palette=PALETA_SUAVE,
    s=250,
    alpha=0.8,
    edgecolor='none',
    legend=False
)

for i, row in model_performance.iterrows():
    plt.text(row['mean'], row['std'] + 0.008, row['modelo_rag'],
             ha='center', va='bottom', fontdict={'size': 9, 'weight': 'bold'})

# plt.title('Análise de Quadrantes: Performance vs. Consistência por RAG', pad=20, fontsize=16)
plt.xlabel('← Baixa Performance | Alta Performance →', fontsize=12)
plt.ylabel('← Baixa Inconsistência (Mais Estável) | Alta Inconsistência →', fontsize=12)

ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Adicionar linhas de referência para a média geral que formam os quadrantes
x_mean = model_performance['mean'].mean()
y_mean = model_performance['std'].mean()
plt.axvline(x_mean, color='gray', linestyle='--', linewidth=1)
plt.axhline(y_mean, color='gray', linestyle='--', linewidth=1)

box_props = dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.7, edgecolor='none')

# Quadrante Superior Esquerdo: Baixa performance, alta inconsistência
plt.text(x_mean - 0.02, y_mean + 0.01, 'Fraco e Instável',
         ha='right', va='bottom', fontsize=10, color='#c70000', style='italic', bbox=box_props)

# Quadrante Superior Direito: Alta performance, alta inconsistência
plt.text(x_mean + 0.02, y_mean + 0.01, 'Bom, mas Instável',
         ha='left', va='bottom', fontsize=10, color='#b35f00', style='italic', bbox=box_props)

# Quadrante Inferior Esquerdo: Baixa performance, baixa inconsistência
plt.text(x_mean - 0.02, y_mean - 0.01, 'Fraco, mas Estável',
         ha='right', va='top', fontsize=10, color='#4a4a4a', style='italic', bbox=box_props)

# Quadrante Inferior Direito: Alta performance, baixa inconsistência (IDEAL)
plt.text(x_mean + 0.02, y_mean - 0.01, 'Ideal:\nForte e Estável',
         ha='left', va='top', fontsize=11, color='#006300', weight='bold', bbox=box_props)

salvar_grafico('./graphs/performance_vs_consistencia_quadrantes.png')


# 10: Análise de Ranking dos RAG's por Performance -------------------------------------------------------------------------------------------------------------------
print("Gerando Gráfico 10: Análise de Ranking dos RAG...") 

# Cria uma coluna 'ranking' para cada grupo de perguntas
df_individual['ranking'] = df_individual.groupby('id_pergunta')['score_final'].rank(method='dense', ascending=False).astype(int)

# Conta a frequência de cada ranking para cada modelo
ranking_counts = df_individual.groupby(['modelo_rag', 'ranking']).size().unstack(fill_value=0)

# Garante que todas as posições de ranking possíveis estejam presentes como colunas
num_models = df_individual['modelo_rag'].nunique()
for i in range(1, num_models + 1):
    if i not in ranking_counts.columns:
        ranking_counts[i] = 0
ranking_counts = ranking_counts[sorted(ranking_counts.columns)]

# Plota o gráfico de barras empilhadas
ranking_counts.plot(
    kind='barh',
    stacked=True,
    figsize=(12, 8),
    color=PALETA_SUAVE,
    edgecolor='white',
    legend=False # Desativamos a legenda automática para ter controle total
)
# 1. TÍTULO MAIS INFORMATIVO
# Um título de duas linhas que explica o que está sendo medido e o total de avaliações.
# plt.title(
#     'Consistência de Desempenho: Frequência de Posições no Ranking\n(Base: 20 Avaliações)',
#     fontsize=14,
#     pad=20
# )

# 2. RÓTULO DO EIXO X MAIS DESCRITIVO
plt.xlabel('Número de vezes que a variante alcançou cada posição')
plt.ylabel('RAG')

# A legenda que discutimos, horizontal e na parte inferior
plt.legend(
    title='Posição no Ranking:', # Adicionei ':' para clareza
    loc='upper center',
    bbox_to_anchor=(0.5, -0.15),
    ncol=6,
    frameon=False
)

ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# As duas linhas a seguir, que criavam a linha vertical, foram removidas.
# plt.axvline(x=10, color='gray', linestyle='--', linewidth=0.8, alpha=0.7)
# plt.text(10.1, -0.5, 'Metade das\navaliações', color='dimgray', ha='left', fontsize=8)


# Adicionar rótulos dentro das barras (código original mantido)
for container in ax.containers:
    labels_values = [f'{int(v)}' if v > 0 else '' for v in container.datavalues]
    ax.bar_label(container, label_type='center', color='white', weight='bold',
                        labels=labels_values)

# Ajuste para garantir que a legenda e os rótulos não sejam cortados
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

salvar_grafico('./graphs/analise_ranking_rags.png')


# 11: Heatmap de Performance (vs. Pergunta) -------------------------------------------------------------------------------------------------------------------
print("Gerando Gráfico 11: Heatmap de Performance com Paleta Suave...")

PALETA_SUAVE_HEATMAP = [
    "#00A8E5",  
    "#D2C0FF",  
    "#F7A9C3",  
    "#A7E0F5",  
    "#DEDEF5", 
    "#F8D0EA" 
]

performance_pivot = df_individual.pivot_table(
    index='modelo_rag',
    columns='id_pergunta',
    values='score_final'
)

plt.figure(figsize=(14, 8))

custom_cmap = sns.light_palette(PALETA_SUAVE_HEATMAP[0], as_cmap=True)

sns.heatmap(
    performance_pivot,
    annot=True,       # Mostra os scores nas células
    cmap=custom_cmap, # Usando o mapa de cores personalizado
    linewidths=.5,
    fmt=".2f",        # Formato com duas casas decimais
    linecolor='white', # Adiciona uma linha branca sutil para separar melhor as células
    cbar_kws={'label': 'Score Final'} # Adiciona um rótulo à barra de cores
)

# plt.title('Heatmap de Performance: Score Final por RAG e Pergunta', pad=20, fontsize=16)
plt.xlabel('ID da Pergunta')
plt.ylabel('RAG')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

salvar_grafico('./graphs/heatmap_performance_pergunta.png')


print(f"Plots salvos: {plot_files}")

Gerando Gráfico 9: Performance vs. Consistência com Quadrantes...
Gerando Gráfico 10: Análise de Ranking dos RAG...
Gerando Gráfico 11: Heatmap de Performance com Paleta Suave...
Plots salvos: ['./graphs/distribuicao_scores_finais.png', './graphs/score_medio_por_rag.png', './graphs/score_medio_por_pergunta.png', './graphs/distribuicao_metricas_llm.png', './graphs/metricas_llm_por_rag.png', './graphs/bertscore_f1_por_comparacao.png', './graphs/contagem_avaliacoes_por_pergunta.png', './graphs/contagem_avaliacoes_por_rag.png', './graphs/performance_vs_consistencia_quadrantes.png', './graphs/analise_ranking_rags.png', './graphs/heatmap_performance_pergunta.png']
